# LLM LoRA fine-tuning

⚠️ **IMPORTANT:** Before running this notebook, copy it into your user directory at `user/<your-username>/`.  
This ensures that any output or intermediate checkpoints are stored in your personal workspace.

Once copied, follow the environment setup instructions in the [README.md](./README.md), and connect this notebook to the `.venv` environment you created during setup.

In [104]:
from finetune import (
    get_device,
    load_and_prepare_dataset,
    tokenize_dataset,
    create_lora_model,
    run_experiment,
)

In [105]:
CONFIGS = {
    "quick": {
        "dataset_name": "rotten_tomatoes",
        "model_name": "roberta-base",
        "max_length": 512,
        "num_labels": 2,
        "num_epochs": 4,
        "batch_size": 8,
        "learning_rate_full": 2e-5,
        "learning_rate_lora": 5e-4,
        "lora_rank": 16,
        "lora_alpha_ratio": 2,
        "description": "Quick experiment: Rotten Tomatoes + RoBERTa-base",
    },
    "full": {
        "dataset_name": "imdb",
        "model_name": "roberta-large",
        "max_length": 512,
        "num_labels": 2,
        "num_epochs": 3,
        "batch_size": 8,
        "learning_rate_full": 2e-5,
        "learning_rate_lora": 3e-4,
        "lora_rank": 16,
        "lora_alpha_ratio": 2,
        "description": "Full experiment: IMDB + RoBERTa-large (GPU recommended)",
    },
}

In [106]:
# choose config
config = CONFIGS["quick"]  # or "full"
device = get_device()
print(f"Running LoRA fine-tuning on {config['dataset_name']} ({config['model_name']})")

Running LoRA fine-tuning on rotten_tomatoes (roberta-base)


In [107]:
# load and tokenize data
dataset, text_field = load_and_prepare_dataset(config["dataset_name"])
tokenized_dataset, tokenizer = tokenize_dataset(
    dataset, text_field, config["model_name"], config["max_length"]
)

In [108]:
# create and train LoRA model
lora_alpha = config["lora_rank"] * config["lora_alpha_ratio"]
model = create_lora_model(
    config["model_name"],
    config["num_labels"],
    rank=config["lora_rank"],
    alpha=lora_alpha,
)

results, model = run_experiment(
    model=model,
    device=device,
    tokenized_dataset=tokenized_dataset,
    config=config,
    experiment_name="LoRA Fine-Tuning",
    learning_rate=config["learning_rate_lora"],
    is_full_finetuning=False,
)

print("\n=== LoRA Fine-Tuning Results ===")
for k, v in results.items():
    print(f"{k:20s}: {v}")

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['heads.default.3.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
There are adapters available but none are activated for the forward pass.
/storage/home/hcoda1/7/yzhang3925/dsgt-arc/fall-2025-interest-group-projects/project/01-llm-lora/.venv/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.462700,0.388686,0.849906
2,0.386800,0.398311,0.883677
3,0.420400,0.530952,0.878987
4,0.392800,0.370052,0.885553


/storage/home/hcoda1/7/yzhang3925/dsgt-arc/fall-2025-interest-group-projects/project/01-llm-lora/.venv/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/storage/home/hcoda1/7/yzhang3925/dsgt-arc/fall-2025-interest-group-projects/project/01-llm-lora/.venv/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/storage/home/hcoda1/7/yzhang3925/dsgt-arc/fall-2025-interest-group-projects/project/01-llm-lora/.venv/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/storage/home/hcoda1/7/yzhang3925/dsgt-arc/fall-2


=== LoRA Fine-Tuning Results ===
experiment_name     : LoRA Fine-Tuning
training_time       : 268.1580820083618
train_loss          : 0.44864546672235267
eval_loss           : 0.37005192041397095
eval_accuracy       : 0.8855534709193246
total_params        : 126617435
trainable_params    : 1971803
trainable_percentage: 1.5572918532112106
